In [1]:
import tensorflow as tf
from tensorflow import keras
from keras import layers, models
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Activation, Conv2, Dense, Dropout
from tensorflow.keras.optimizers import Adagrad, Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
import tensorflow_hub as hub

import cv2
import glob
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import scipy

from PIL import Image


# データを正方形にリサイズするときの辺の長さ
square_length = 64

# 画像を置いてるパス
image_path = "../image"

# ラベル付け(0春 1夏 2冬)
# TODO: ? フォルダ名に数字入れておくとかのほうがいいかもしれない？
# TODO: 3つ以上に増やす場合は、class_modeやlossをbinaryからcategricalに変える
def get_label(value):
    if value == "Ariel":
        return 0
    elif value == "Other":
        return 1
    
# ラベル更新したら一緒に更新する
def max_label():
    return 2

end


In [3]:
# 画像の読み込み
datagen = ImageDataGenerator(
    rescale=1./255
#     validation_split=0.1
    )

train_generator = datagen.flow_from_directory(
    "../image/eye/train",
    target_size=(square_length, square_length),
    class_mode='binary',
    batch_size=32,
    subset='training',
)

Found 681 images belonging to 2 classes.


In [4]:
print("start")

# モデルの構築
model = keras.Sequential([
#     keras.layers.Conv2D(32, (8,8), activation="relu"),
#     keras.layers.Conv2D(32, (8,8), activation="relu"),
#     keras.layers.MaxPooling2D(pool_size=(4,4)),
#     keras.layers.Flatten(),
#     keras.layers.Dropout(0.2),
#     keras.layers.Dense(200, activation="relu"),
#     keras.layers.Dropout(0.4),
#     keras.layers.Dense(200, activation="relu"),
#     keras.layers.Dropout(0.4),
#     keras.layers.Dense(1),
#     keras.layers.Activation("softmax")
    hub.KerasLayer(
        "https://tfhub.dev/tensorflow/efficientnet/b4/feature-vector/1",
        trainable=False,
    ),
    keras.layers.Dense(1, activation="sigmoid")
])
# memo: shapeで警告出るのは、keras.layers.reshape((...)),imput_shape=(...))を使えば良さそう

# # モデルをコンパイル
model.compile(loss="binary_crossentropy", optimizer=Adam(learning_rate=0.001), metrics=["accuracy"])
# 学習を実行
model.fit(train_generator, steps_per_epoch=1, epochs=30)
print(model.summary())

start
Epoch 1/30
1/1 [==============================] - 10s 10s/step - loss: 0.6691 - accuracy: 0.6562
Epoch 2/30
1/1 [==============================] - 0s 56ms/step - loss: 0.6281 - accuracy: 0.6667
Epoch 3/30
1/1 [==============================] - 0s 151ms/step - loss: 0.6450 - accuracy: 0.6562
Epoch 4/30
1/1 [==============================] - 0s 136ms/step - loss: 0.5841 - accuracy: 0.7188
Epoch 5/30
1/1 [==============================] - 0s 137ms/step - loss: 0.6732 - accuracy: 0.5938
Epoch 6/30
1/1 [==============================] - 0s 134ms/step - loss: 0.6797 - accuracy: 0.5625
Epoch 7/30
1/1 [==============================] - 0s 135ms/step - loss: 0.6252 - accuracy: 0.6562
Epoch 8/30
1/1 [==============================] - 0s 135ms/step - loss: 0.6178 - accuracy: 0.6562
Epoch 9/30
1/1 [==============================] - 0s 135ms/step - loss: 0.5798 - accuracy: 0.7812
Epoch 10/30
1/1 [==============================] - 0s 137ms/step - loss: 0.5926 - accuracy: 0.6875
Epoch 11/30
1/1

In [16]:
# テスト用ディレクトリの画像でチェック。正解率を表示する。
total = 0.
ok_count = 0.

test_path = "../image/eye/test"
for dir in os.listdir(test_path):
    if dir == ".DS_Store":
        continue

    dir_path = test_path + "/" + dir
    label = get_label(dir)

    for file in os.listdir(dir_path):
        if file != ".DS_Store":
            file_path = dir_path + "/" + file
            image = np.array(Image.open(file_path).resize(size=(square_length, square_length)))
#             image = image.transpose(2, 0, 1)
#             image = image.reshape(image.shape[0] * image.shape[1] * image.shape[2], 1)
            result = model.predict(np.array([image / 255.]))
            print("[" + str(int(total)) + "]label:", label, "result:", result)

            total += 1.

            if label - result > 0.:
                ok_count += 1.
                print("ok")

print("seikai: ", ok_count / total * 100, "%")

[0]label: 0 result: [[0.59576255]]
[1]label: 0 result: [[0.66527796]]
[2]label: 0 result: [[0.5302289]]
[3]label: 0 result: [[0.57677734]]
[4]label: 0 result: [[0.6089865]]
[5]label: 0 result: [[0.57276654]]
[6]label: 0 result: [[0.513666]]
[7]label: 0 result: [[0.6260319]]
[8]label: 1 result: [[0.75651133]]
ok
[9]label: 1 result: [[0.79491585]]
ok
[10]label: 1 result: [[0.7936898]]
ok
[11]label: 1 result: [[0.7131064]]
ok
[12]label: 1 result: [[0.5666309]]
ok
[13]label: 1 result: [[0.6289366]]
ok
[14]label: 1 result: [[0.6926516]]
ok
[15]label: 1 result: [[0.692444]]
ok
seikai:  50.0 %
